In [6]:
import pysam
from tools.intervals import *
from collections import *

In [10]:
bam = 'A1.consensus.bam'
intervals = []
for x in open('/hive/users/ifiddes/notch2nl_berkeley_data/new_consensus/NOTCH2NL-consensus-exons.bed'):
    x = x.split()
    intervals.append(ChromosomeInterval(x[0], int(x[1]), int(x[2]), '+'))

In [53]:
from glob import glob
bams = glob('*/*bam')

In [55]:
txs = {}
for b in bams:
    nucs = {"A", "T", "G", "C", "a", "t", "g", "c"}
    exons = []
    for i in intervals:
        bases = []
        pileup = pysam.mpileup('-r', '{}:{}-{}'.format(i.chromosome, i.start, i.stop - 1), b)
        pileup = pileup.split('\n')
        for col in pileup:
            col = col.split()
            if len(col) != 6:
                continue
            col_bases = Counter(x.upper() for x in col[4] if x in nucs)
            if len(col_bases) == 0:
                continue
            bases.append(col_bases.most_common(1)[0][0])
        exons.append(bases)
    name = os.path.basename(b).split('.')[0]
    txs[name] = ''.join([''.join(x) for x in exons])

In [57]:
from tools.bio import *
with open('h9-transcript-assemblies.fa', 'w') as outf:
    for name, seq in txs.iteritems():
        write_fasta(outf, name, seq)

In [56]:
print len(txs)

10
